In [1]:
import os
import csv
import pandas as pd
import numpy as np

# import Well Location(geospatial info) csv files as Pandas dataframes

In [2]:
# Specify the columns you want to read
columns_to_read = ['WCRNUMBER', 'DECIMALLATITUDE', 'DECIMALLONGITUDE','BOTTOMOFPERFORATEDINTERVAL',\
            'TOTALCOMPLETEDDEPTH','TOPOFPERFORATEDINTERVAL','WELLYIELD','TESTTYPE','TOTALDRAWDOWN',\
            'PUMPTESTLENGTH','PUMPTESTLENGTH','STATICWATERLEVEL','CASINGDIAMETER', ]

try:
    # Read CSV file with specified columns
    df1 = pd.read_csv("wellcompletionreports.csv", usecols=columns_to_read)
    
    # Display the first few rows to verify
    print(df1.head())
except FileNotFoundError:
    print("File not found.")
except Exception as e:
    print(f"Error: {e}")

    WCRNUMBER  DECIMALLATITUDE  DECIMALLONGITUDE  TOTALCOMPLETEDDEPTH  \
0  WCR0101697          37.5279          -122.051                134.0   
1  WCR0100841          37.6725          -121.706                  NaN   
2  WCR0104532          37.6581          -121.887                  NaN   
3  WCR0102316          37.7161          -121.941                  NaN   
4  WCR0102712          37.5279          -122.033                  NaN   

   TOPOFPERFORATEDINTERVAL  BOTTOMOFPERFORATEDINTERVAL  CASINGDIAMETER  \
0                      NaN                         NaN             6.0   
1                      NaN                         NaN             NaN   
2                      NaN                         NaN            14.0   
3                      NaN                         NaN             NaN   
4                      NaN                         NaN             NaN   

   STATICWATERLEVEL  TOTALDRAWDOWN TESTTYPE  PUMPTESTLENGTH  WELLYIELD  
0              18.8            NaN      NaN

## Convert '/' character to decimal character
# i.e.   37/41/13.06/  to 37+41  divide by /60  + 13.06 # divide by 3600

In [3]:
def convert_latitude(lat):
    if isinstance(lat, str) and '/' in lat:
        lat_parts = lat.split('/')[:-1]  # Exclude the last element
        try:
            degrees, minutes, seconds = map(float, lat_parts)
            decimal_lat = degrees + (minutes / 60) + (seconds / 3600)
            return decimal_lat
        except ValueError:
            return None
    else:
        try:
            return float(lat)
        except (ValueError, TypeError):
            return None

def convert_longitude(lon):
    if isinstance(lon, str) and '/' in lon:
        lon_parts = lon.split('/')[:-1]  # Exclude the last element
        try:
            degrees, minutes, seconds = map(float, lon_parts)
            decimal_lon = degrees + (minutes / 60) + (seconds / 3600)
            return decimal_lon
        except ValueError:
            return None
    else:
        try:
            return float(lon)
        except (ValueError, TypeError):
            return None

In [4]:
df1['Decimal_Lat'] = df1['DECIMALLATITUDE'].apply(convert_latitude)
df1['Decimal_Long'] = df1['DECIMALLONGITUDE'].apply(convert_longitude)

In [5]:
df1 = df1.dropna(subset=['Decimal_Long','Decimal_Lat','TOTALDRAWDOWN' ,'WELLYIELD'])
df1 = df1.drop(columns=['DECIMALLONGITUDE','DECIMALLATITUDE' ])
# print(df1.head())

In [6]:
# Convert columns to numeric, setting errors='coerce' to convert invalid parsing to NaN
df1['TOTALDRAWDOWN'] = pd.to_numeric(df1['TOTALDRAWDOWN'], errors='coerce')
df1['TOTALCOMPLETEDDEPTH'] = pd.to_numeric(df1['TOTALCOMPLETEDDEPTH'], errors='coerce')

# Create the 'drawdown_flag' column based on the condition
df1['drawdown_flag'] = (df1['TOTALDRAWDOWN'] > df1['TOTALCOMPLETEDDEPTH']) & \
                       (~df1['TOTALCOMPLETEDDEPTH'].isna()) & \
                       (df1['TOTALCOMPLETEDDEPTH'] != df1['TOTALDRAWDOWN'])

# Print 'not reliable' if any value in 'drawdown_flag' is True
if df1['drawdown_flag'].any():
    print("not reliable")

# Dropping rows where 'drawdown_flag' is True
df1 = df1[df1['drawdown_flag'] != True]

# Display the DataFrame
df1.head()

not reliable


,WCRNUMBER,TOTALCOMPLETEDDEPTH,TOPOFPERFORATEDINTERVAL,BOTTOMOFPERFORATEDINTERVAL,CASINGDIAMETER,STATICWATERLEVEL,TOTALDRAWDOWN,TESTTYPE,PUMPTESTLENGTH,WELLYIELD,Decimal_Lat,Decimal_Long,drawdown_flag
402,WCR1998-000135,370.0,100.0,310.0,5.0,0.0,310.0,NaN,2.0,2.0,37.5862,-121.671,False
618,WCR2020-009133,NaN,NaN,NaN,NaN,5.0,0.5,Pump,1.0,50.0,37.5532,-122.060,False
662,WCR2022-003003,340.0,200.0,340.0,NaN,80.0,200.0,Pump,4.0,15.0,37.6505,-121.652,False
1028,WCR1998-003767,522.0,374.0,424.0,8.0,104.0,25.0,NaN,8.0,380.0,37.7018,-121.814,False
1038,WCR1996-001797,77.0,47.0,67.0,6.0,NaN,20.0,NaN,0.5,0.5,37.4731,-121.889,False


In [7]:
df2 = pd.read_csv("C://Users//betebari//Documents//C2VSim_Texture//OSWCR//UPDATED_wellcompletionreports.csv")

# Merge df1 and df2 on 'WCRNUMBER' to separate Central Valley pump test from California database
merged_df = pd.merge(df2, df1, on='WCRNUMBER', how='inner')

# Display the DataFrame
merged_df.head()

,Unnamed: 0,WCRNUMBER,DECIMALLAT,DECIMALLON,UTMX_y,UTMY_y,TOTALCOMPLETEDDEPTH,TOPOFPERFORATEDINTERVAL,BOTTOMOFPERFORATEDINTERVAL,CASINGDIAMETER,STATICWATERLEVEL,TOTALDRAWDOWN,TESTTYPE,PUMPTESTLENGTH,WELLYIELD,Decimal_Lat,Decimal_Long,drawdown_flag
0,6,WCR2019-014989,34.8435,-118.901,874863.218055,3.863357e+06,600.0,200.0,600.0,NaN,180.0,200.0,Pump,24.0,5.0,34.8435,-118.901,False
1,213,WCR2022-007916,35.0127,-118.886,875462.353961,3.882193e+06,1445.0,NaN,NaN,NaN,715.0,52.0,Pump,12.0,2750.0,35.0127,-118.886,False
2,330,WCR2022-002164,35.0479,-119.123,853669.007339,3.885233e+06,2580.0,NaN,NaN,NaN,689.0,177.0,Pump,12.0,2250.0,35.0479,-119.123,False
3,331,WCR2022-006536,35.0482,-119.104,855401.837406,3.885334e+06,2560.0,NaN,NaN,NaN,699.0,81.0,Pump,12.0,2250.0,35.0482,-119.104,False
4,548,WCR2020-001133,35.0722,-118.642,897456.225638,3.889745e+06,580.0,480.0,580.0,NaN,141.0,0.0,Air Lift,4.0,40.0,35.0722,-118.642,False


#Estimate of Transmisivity using Driscoll approach (1986, units gpd/ft, best if aquifer is Confined)
#Using equation (1), Driscoll (1986) assumed the following values to develop approximate formulas for estimating transmissivity from specific capacity in confined and unconfined aquifers:

#T = 30,000 US gal/day/ft

#S = 0.001 (confined) or 0.075 (unconfined)

#rw = 0.5 ft

#t = 1 day

#T = 2000(Q/sw) confined aquifer

#T = 1500(Q/sw) unconfined aquifer

In [8]:
# Assign merged_df to df1
df1 = merged_df

In [9]:
# Convert columns to numeric, setting errors='coerce' to convert invalid parsing to NaN
df1['WELLYIELD'] = pd.to_numeric(df1['WELLYIELD'], errors='coerce')
df1['BOTTOMOFPERFORATEDINTERVAL'] = pd.to_numeric(df1['BOTTOMOFPERFORATEDINTERVAL'], errors='coerce')
df1['TOPOFPERFORATEDINTERVAL'] = pd.to_numeric(df1['TOPOFPERFORATEDINTERVAL'], errors='coerce')
df1['TOTALCOMPLETEDDEPTH'] = pd.to_numeric(df1['TOTALCOMPLETEDDEPTH'], errors='coerce')
df1['STATICWATERLEVEL'] = pd.to_numeric(df1['STATICWATERLEVEL'], errors='coerce')
df1['CASINGDIAMETER'] = pd.to_numeric(df1['CASINGDIAMETER'], errors='coerce')
df1['PUMPTESTLENGTH'] = pd.to_numeric(df1['PUMPTESTLENGTH'], errors='coerce')

#  calculate c using:
df1['SpecificCapacity'] = df1['WELLYIELD'] / df1['TOTALDRAWDOWN']

# Set SpecificCapacity to NaN where it is greater than 200
df1.loc[df1['SpecificCapacity'] > 200, 'SpecificCapacity'] = pd.NA

# Dropping rows where 'SpecificCapacity' is Nan
df1 = df1.dropna(subset=['SpecificCapacity'])

# Dropping rows where 'SpecificCapacity' is less than 0.01
df1 = df1[df1['SpecificCapacity'] > 0.01]

# Round down the specified columns to the nearest 2 decimal places
df1['SpecificCapacity'] = df1['SpecificCapacity'].apply(lambda x: np.floor(x * 100) / 100)

# Remove inf and -inf values, replacing them with NaN
df1['SpecificCapacity'] = df1['SpecificCapacity'].apply(lambda x: x if np.isfinite(x) else np.nan)

In [10]:
# calculate Saturated thickness using top and bottom of screen
df1['SaturatedThickness'] = df1['TOTALCOMPLETEDDEPTH'] -df1['STATICWATERLEVEL'] 

# Replace nan TOTAL DRAWDOWN values with 1
df1['TOTALDRAWDOWN'] = np.where(df1['TOTALDRAWDOWN'] <= 0, np.nan, df1['TOTALDRAWDOWN']+1)

# Replace negative values with NaN for Saturated Thickness
df1['SaturatedThickness'] = np.where(df1['SaturatedThickness'] < 0, np.nan, df1['SaturatedThickness'])

# Where ScreenedInterval is NaN, calculate it using TOTALCOMPLETEDDEPTH and STATICWATERLEVEL
df1.loc[df1['SaturatedThickness'].isna(), 'SaturatedThickness'] = df1['BOTTOMOFPERFORATEDINTERVAL'] - df1['TOPOFPERFORATEDINTERVAL']

# Dropping rows where 'SpecificCapacity' is less than 0
df1 = df1[df1['SaturatedThickness'] > 0]

# Computing T in m2/day  (Cooper and Jacob (1946) ):¶

In [11]:
from scipy.optimize import minimize

# Define the objective function based on Equation (2)
def cooper_jacob(params, Q, r, t, s_obs):
    T, S = params
    s_calc = (Q / (4 * np.pi * T)) * np.log(2.25 * T * t / r**2 / S)
    return np.sum((s_obs - s_calc)**2)

In [12]:
# Create columns for estimated T and S in the existing df1 DataFrame
df1['Estimated_Tr_Jacob_m2d'] = np.nan
df1['Estimated_Storage_Coefficient'] = np.nan

# Loop through each row in the DataFrame
for idx, row in df1.iterrows():
    Q = row['WELLYIELD']* 5.450992992  # Pumping rate in cubic meters per day
    t = row['PUMPTESTLENGTH']/24  # Time in days
    s_obs = row['TOTALDRAWDOWN']* 0.3048  # Observed drawdown in meters
    
    # Handle missing casing diameter by assuming a value of 12 inches(0.3048 meters)
    # Radius of the well (assumed to be casing diameter)
    r = row['CASINGDIAMETER']  if not np.isnan(row['CASINGDIAMETER']* 0.0254) else 0.3048
    
    # Check if any necessary value is missing
    if np.isnan(Q) or np.isnan(r) or np.isnan(t) or np.isnan(s_obs):
        continue

    # Initial guess for T and S
    initial_guess = [1000, 0.0001]

    # Minimize the objective function using successive approximation
    result = minimize(cooper_jacob, initial_guess, args=(Q, r, t, s_obs), method='Nelder-Mead')

    if result.success:
        T_estimated, S_estimated = result.x
        df1.at[idx, 'Estimated_Tr_Jacob_m2d'] = T_estimated
        df1.at[idx, 'Estimated_Storage_Coefficient'] = S_estimated
    else:
        print(f"Optimization failed for well {row['WCRNUMBER']}")

C:\Users\betebari\AppData\Local\Temp\ipykernel_26584\3922220900.py:6: RuntimeWarning: invalid value encountered in log
  s_calc = (Q / (4 * np.pi * T)) * np.log(2.25 * T * t / r**2 / S)
C:\Users\betebari\AppData\Local\Temp\ipykernel_26584\3922220900.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  s_calc = (Q / (4 * np.pi * T)) * np.log(2.25 * T * t / r**2 / S)
C:\Users\betebari\AppData\Local\Temp\ipykernel_26584\3922220900.py:6: RuntimeWarning: divide by zero encountered in log
  s_calc = (Q / (4 * np.pi * T)) * np.log(2.25 * T * t / r**2 / S)
C:\Users\betebari\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:836: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):


Optimization failed for well WCR2021-013233


In [13]:
# Unit conversion from m2/day to ft2/day
df1['Estimated_Tr_Jacob_ft2d'] = df1['Estimated_Tr_Jacob_m2d']*10.7639   # ft2/day 

# Estimate of Hydraluic Conductivity using jacob approach
df1['Kxy_Jacob_ftd'] = df1['Estimated_Tr_Jacob_ft2d'] /df1['SaturatedThickness']  # ft/day

# Round down the specified columns to the nearest 2 decimal places
df1['Kxy_Jacob_ftd'] = df1['Kxy_Jacob_ftd'].apply(lambda x: np.floor(x * 100) / 100)

# Display the DataFrame
df1.head()

,Unnamed: 0,WCRNUMBER,DECIMALLAT,DECIMALLON,UTMX_y,UTMY_y,TOTALCOMPLETEDDEPTH,TOPOFPERFORATEDINTERVAL,BOTTOMOFPERFORATEDINTERVAL,CASINGDIAMETER,...,WELLYIELD,Decimal_Lat,Decimal_Long,drawdown_flag,SpecificCapacity,SaturatedThickness,Estimated_Tr_Jacob_m2d,Estimated_Storage_Coefficient,Estimated_Tr_Jacob_ft2d,Kxy_Jacob_ftd
0,6,WCR2019-014989,34.8435,-118.901,874863.218055,3.863357e+06,600.0,200.0,600.0,NaN,...,5.0,34.8435,-118.901,False,0.02,420.0,0.401669,0.000115,4.323526,0.01
1,213,WCR2022-007916,35.0127,-118.886,875462.353961,3.882193e+06,1445.0,NaN,NaN,NaN,...,2750.0,35.0127,-118.886,False,52.88,730.0,1393.462719,0.000108,14999.093365,20.54
2,330,WCR2022-002164,35.0479,-119.123,853669.007339,3.885233e+06,2580.0,NaN,NaN,NaN,...,2250.0,35.0479,-119.123,False,12.71,1891.0,311.637509,0.000113,3354.434983,1.77
3,331,WCR2022-006536,35.0482,-119.104,855401.837406,3.885334e+06,2560.0,NaN,NaN,NaN,...,2250.0,35.0482,-119.104,False,27.77,1861.0,711.974493,0.000104,7663.622240,4.11
6,568,WCR2016-002959,35.0773,-119.109,854819.296323,3.888545e+06,1480.0,540.0,1460.0,NaN,...,800.0,35.0773,-119.109,False,3.33,1070.0,68.182939,0.000111,733.914342,0.68


# computing T in m2/day are (Driscoll 1986):¶
#If we express Q in m3/day and sw in m, the equations for computing T in m2/day are (Batu 1998):

#T = 2000(Q/sw) confined aquifer

#T = 1500(Q/sw) unconfined aquifer

#where T is transmissivity [US gal/day/ft], 
#Q is constant discharge rate [US gal/min] and
#sw is drawdown in the pumped well after 1 day [ft].

In [14]:
#  Estimate of Transmisivity using Driscoll approach units (gpd/ft)
df1['Tr_gpd_ft_Driscoll_Confined'] = df1['SpecificCapacity'] *2000   # gpd/ft
df1['Tr_gpd_ft_Driscoll_Unconfined'] = df1['SpecificCapacity'] *1500  # gpd/ft

# For unit conversion I used https://www.groundwatersoftware.com/calculator_3_unit_conversion.htm
# I found 1 gpd/ft2 = 0.13368055419447  or  {0.13387755 ????}
df1['Tr_ft2_day_Driscoll_Confined'] = df1['Tr_gpd_ft_Driscoll_Confined'] * 0.13368055419447    # ft2/day
df1['Tr_ft2_day_Driscoll_Unconfined'] = df1['Tr_gpd_ft_Driscoll_Unconfined'] * 0.13368055419447  # ft2/day

# Round down the specified columns to the nearest 2 decimal places
df1['Tr_gpd_ft_Driscoll_Confined'] = df1['Tr_gpd_ft_Driscoll_Confined'].apply(lambda x: np.floor(x * 100) / 100)
df1['Tr_gpd_ft_Driscoll_Unconfined'] = df1['Tr_gpd_ft_Driscoll_Unconfined'].apply(lambda x: np.floor(x * 100) / 100)
df1['Tr_ft2_day_Driscoll_Confined'] = df1['Tr_ft2_day_Driscoll_Confined'].apply(lambda x: np.floor(x * 100) / 100)
df1['Tr_ft2_day_Driscoll_Unconfined'] = df1['Tr_ft2_day_Driscoll_Unconfined'].apply(lambda x: np.floor(x * 100) / 100)

# Estimate of Hydraluic Conductivity using Driscoll approach
df1['Kxy_ft_d_Driscoll_Confined'] = df1['Tr_ft2_day_Driscoll_Confined'] /df1['SaturatedThickness']
df1['Kxy_ft_d_Driscoll_Unconfined'] = df1['Tr_ft2_day_Driscoll_Unconfined'] /df1['SaturatedThickness']

# Round down the specified columns to the nearest 2 decimal places
df1['Kxy_ft_d_Driscoll_Confined'] = df1['Kxy_ft_d_Driscoll_Confined'].apply(lambda x: np.floor(x * 100) / 100)
df1['Kxy_ft_d_Driscoll_Unconfined'] = df1['Kxy_ft_d_Driscoll_Unconfined'].apply(lambda x: np.floor(x * 100) / 100)

# Display the DataFrame
# print(df1.head())

# computing T in m2/day are (Batu 1998):

#If we express Q in m3/day and sw in m, the equations for computing T in m2/day are (Batu 1998):

#T = 1.385(Q/sw)    confined aquifer

#T = 1.042(Q/sw)    unconfined aquifer

In [15]:
# unit conversion 
df1['WELLYIELD_m3d'] = df1['WELLYIELD'] * 5.45099296896
df1['TOTALDRAWDOWN_m'] = df1['TOTALDRAWDOWN'] * 0.3048
df1['SaturatedThickness_m'] = df1['SaturatedThickness'] * 0.3048

df1['SpecificCapacity_m'] = df1['WELLYIELD_m3d'] / df1['TOTALDRAWDOWN_m']

df1['Tr_m2_day_Batu_Confined'] = df1['SpecificCapacity_m'] * 1.385 # m2/day
df1['Tr_m2_day_Batu_Unconfined'] = df1['SpecificCapacity_m'] * 1.042 # m2/day

# Round down the specified columns to the nearest 2 decimal places
df1['Tr_m2_day_Batu_Confined'] = df1['Tr_m2_day_Batu_Confined'].apply(lambda x: np.floor(x * 100) / 100)
df1['Tr_m2_day_Batu_Unconfined'] = df1['Tr_m2_day_Batu_Unconfined'].apply(lambda x: np.floor(x * 100) / 100)


# Estimate of Hydraluic Conductivity using Batu approach m/day
df1['Kxy_m_day_Batu_Confined'] = df1['Tr_m2_day_Batu_Confined'] /df1['SaturatedThickness_m']  # m/day
df1['Kxy_m_day_Batu_Unonfined'] = df1['Tr_m2_day_Batu_Unconfined'] /df1['SaturatedThickness_m'] # m/day

# Round down the specified columns to the nearest 2 decimal places
df1['Kxy_m_day_Batu_Confined'] = df1['Kxy_m_day_Batu_Confined'].apply(lambda x: np.floor(x * 100) / 100)
df1['Kxy_m_day_Batu_Unonfined'] = df1['Kxy_m_day_Batu_Unonfined'].apply(lambda x: np.floor(x * 100) / 100)

# Display the DataFrame
# print(df1.head())

# Transmissivity in fractured bedrock  using Mace(1997) (m2/day)

In [16]:
df1['Tr_m2_day_Mace_fracture'] =0.76 * ((df1['SpecificCapacity_m']) ** 1.08) # m2/day

# Estimate of Hydraluic Conductivity using mace approach m/day
df1['Kxy_m_day_Mace_fracture'] = df1['Tr_m2_day_Mace_fracture'] /df1['SaturatedThickness_m']

# Round down the specified columns to the nearest 2 decimal places 
df1['Tr_m2_day_Mace_fracture'] = df1['Tr_m2_day_Mace_fracture'].apply(lambda x: np.floor(x * 100) / 100)
df1['Kxy_m_day_Mace_fracture'] = df1['Kxy_m_day_Mace_fracture'].apply(lambda x: np.floor(x * 100) / 100)

# Display the DataFrame
# print(df1.head())

# Transmisivity Razack & Huntley (m2/day)

In [17]:
df1['Tr_Razack_m2d'] =15.3 * ((df1['SpecificCapacity_m'] ** 0.67))
df1['Tr_Razack_ft2d'] =33.6 * (((192.5 * df1['SpecificCapacity'])) ** 0.67)

# Round down the specified columns to the nearest 2 decimal places
df1['Tr_Razack_m2d'] = df1['Tr_Razack_m2d'].apply(lambda x: np.floor(x * 100) / 100)
df1['Tr_Razack_ft2d'] = df1['Tr_Razack_ft2d'].apply(lambda x: np.floor(x * 100) / 100)

# Estimate of Hydraluic Conductivity using mace approach
df1['Kxy_Razack_md'] = df1['Tr_Razack_m2d'] /df1['SaturatedThickness_m']
df1['Kxy_Razack_ftd'] = df1['Tr_Razack_m2d'] /df1['SaturatedThickness']

# Round down the specified columns to the nearest 2 decimal places
df1['Kxy_Razack_md'] = df1['Kxy_Razack_md'].apply(lambda x: np.floor(x * 100) / 100)
df1['Kxy_Razack_ftd'] = df1['Kxy_Razack_ftd'].apply(lambda x: np.floor(x * 100) / 100)

# Display the DataFrame
df1.head()

,Unnamed: 0,WCRNUMBER,DECIMALLAT,DECIMALLON,UTMX_y,UTMY_y,TOTALCOMPLETEDDEPTH,TOPOFPERFORATEDINTERVAL,BOTTOMOFPERFORATEDINTERVAL,CASINGDIAMETER,...,Tr_m2_day_Batu_Confined,Tr_m2_day_Batu_Unconfined,Kxy_m_day_Batu_Confined,Kxy_m_day_Batu_Unonfined,Tr_m2_day_Mace_fracture,Kxy_m_day_Mace_fracture,Tr_Razack_m2d,Tr_Razack_ft2d,Kxy_Razack_md,Kxy_Razack_ftd
0,6,WCR2019-014989,34.8435,-118.901,874863.218055,3.863357e+06,600.0,200.0,600.0,NaN,...,0.61,0.46,0.00,0.00,0.31,0.00,8.89,82.90,0.06,0.02
1,213,WCR2022-007916,35.0127,-118.886,875462.353961,3.882193e+06,1445.0,NaN,NaN,NaN,...,1285.18,966.90,5.77,4.34,1218.23,5.47,1489.11,16274.70,6.69,2.03
2,330,WCR2022-002164,35.0479,-119.123,853669.007339,3.885233e+06,2580.0,NaN,NaN,NaN,...,313.09,235.55,0.54,0.40,265.07,0.45,578.12,6261.60,1.00,0.30
3,331,WCR2022-006536,35.0482,-119.104,855401.837406,3.885334e+06,2560.0,NaN,NaN,NaN,...,679.64,511.32,1.19,0.90,612.22,1.07,971.73,10570.69,1.71,0.52
6,568,WCR2016-002959,35.0773,-119.109,854819.296323,3.888545e+06,1480.0,540.0,1460.0,NaN,...,82.22,61.85,0.25,0.18,62.55,0.19,236.02,2552.38,0.72,0.22


In [18]:
# Save the updated DataFrame to a new CSV file
output_file = 'Estimates_of_Transmissivity_from_Specific_Capacity.csv'
df1.to_csv(output_file, index=False)

print(f"Transmissivity CSV file saved as '{output_file}'")

Transmissivity CSV file saved as 'Estimates_of_Transmissivity_from_Specific_Capacity.csv'
